# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:   Cristian Muñiz Álvarez <br>
Url: https://github.com/cmuniz/Algoritmos-de-Optimizacion/blob/1438bc74df5bce0a9987edd1405b16cd54553d88/sesiones-doblaje.ipynb<br>

Problema:

# Organizar sesiones de doblaje <br>


### Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. 

Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. 

El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. 

Los datos son:
- Números de actores: 10
- Números de tomas: 30
- Actores/Tomas: https://bit.ly/36D8IuK

## ¿Cuántas posibilidades hay sin tener en cuenta las restricciones?

Sin tener en cuenta las restricciones, es decir, sin importar cuántas tomas por grupo, ni actores. Bajo esta condición, lo que tenemos que contar es de cuántas formas se pueden agrupar las 30 tomas en subconjuntos no vacíos. Este es un problema de partición de un conjunto.

La formas de particionar un conjunto de $n$ elementos en subjuntos disjuntos no vacíos es mediante el número de Bell. Para calcular este número fácilmente para nuestro conjunto $n=30$, podemos utilizar la librería `sympy` de python:

In [1]:
from sympy import bell 
numero = int(bell(30))

print(f"Número de particiones: {numero} ~ {numero:.1E}")

Número de particiones: 846749014511809332450147 ~ 8.5E+23


Sin tener en cuenta que cada grupo debe tener como máximo 6 tomas, tenemos un espacio de búsqueda del orden ~$10^{23}$ elementos.

## Estructura de datos del problema

La estructura de datos planteada para el problema es la siguiente:
- **Calendario:** se representa mediante una lista de listas. Permite ver qué tomas hay por día, se recorre facilmente y permite verificar si se puede agregar una toma a ese día o si se ha llegado al máximo.
```python
  dias = [
    [1, 5, 9],   # Día 1: tomas
    [2, 4],      # Día 2: tomas
    ...
    ]
```

- **Tomas:** las tomas se representan con un diccionario donde la clave identifica la toma y el valor es un conjunto con los actores que participan en ella. De esta forma se puede consultar fácilmente qué actores participan en cada toma.

```python
  tomas_actores = {
    1: {1, 3, 4},
    2: {2, 4},
    ...
}
```

- **Tomas restantes:** Se representa mediante un conjunto que permite un buen control sobre los elementos pendientes y tiene un buen rendimiento.
  
  ```python
  tomas_restantes = {3, 6, 8, 10}
   ```


## Función objetivo del Problema

El objetivo principal es **minimizar** el coste total asociado a la participación de los actores durante el rodaje. 

Según el enunciado, se indica que cada actor cobra una vez por cada día que asiste al rodaje, independientemente del número de tomas que realice ese día.
Por tanto, el coste total del calendario de grabación es la suma, para cada día, del número de actores distintos que deben asistir.


Formalmente, se define una variable binaria $y_{a,d}$ que vale 1 si el actor $a$ trabaja el día $d$, y 0 en caso contrario. La función objetivo a minimizar es:

$$
\min \sum_{a \in A} \sum_{d \in D} y_{a,d}
$$
donde $A$ es el conjunto de actores y $D$ el conjunto de días que se requieran para realizar todas las tomas.


Implementación de la función de coste en Python:

In [2]:
def calcular_coste(calendario):
    coste = 0
    for dia in calendario:
        actores = set()
        for toma in dia:
            actores.update(tomas_actores[toma])
        coste += len(actores)
    return coste

## Algoritmo por Fuerza Bruta

Resolver el problema mediante fuerza bruta significa generar todas las posibles particiones del número de tomas en grupos con el máximo de tomas por día, calcular el coste mediante la función anterior, y quedarnos con la que tenga menor coste total de entre todas.

Dado que la función de coste ya la tenemos, dividimos el algoritmo en dos funciones: la función propia del agoritmo de fuerza bruta y una función para generar todas las particiones.

- **generar_particiones:** Contruye todas las perticiones posibles, respetando el límite de tomas por día, tal que:
  1. Si no quedan tomas por asignar, se devuelve una lista vacía.
  2. Se generan todos los grupos posibles de 1 hasta el máximo de tomas por día.
  3. Para cada grupo se calcula el conjunto de tomas restantes y se aplica recursivamente la función sobre ellas.
- **fuerza_bruta:** Recorre todas las particiones generadas, calcula el coste de cada una y devuelve la mejor solución encontrada.

In [3]:
import itertools
import copy

In [4]:
# Generador de todas las particiones posibles respetando el límite por día
def generar_particiones(tomas, max_tomas_dia):
    if not tomas:
        yield []
        return
    for i in range(1, min(len(tomas), max_tomas_dia) + 1):
        for grupo in itertools.combinations(tomas, i):
            restantes = tomas - set(grupo)
            for subpart in generar_particiones(restantes, max_tomas_dia):
                yield [list(grupo)] + subpart

In [5]:
def fuerza_bruta(tomas_actores):
    todas_tomas = set(tomas_actores.keys())
    mejor_coste = float('inf')
    mejor_plan = None

    for calendario in generar_particiones(todas_tomas, MAX_TOMAS_POR_DIA):
        coste = calcular_coste(calendario)
        if coste < mejor_coste:
            mejor_coste = coste
            mejor_plan = calendario
    return mejor_coste, mejor_plan

### Conjunto de datos de prueba

Dado que el espacio de busqueda crece exponencialmente. Usamos un conjuto más pequeño para probar este algoritmo:

In [6]:
tomas_actores = {
    1: {1, 2},
    2: {2, 3},
    3: {3, 4},
    4: {4, 5},
    5: {1, 5},
    6: {2, 4},
    7: {1, 3},
    8: {2, 5},
    9: {3, 5},
}

MAX_TOMAS_POR_DIA = 3
NUM_ACTORES = 5

In [7]:
%time mejor_coste, mejor_plan = fuerza_bruta(tomas_actores)

CPU times: total: 21.1 s
Wall time: 21.4 s


In [8]:
print("Costo mínimo optimizado:", mejor_coste)
for i, dia in enumerate(mejor_plan, start=1):
    actores_dia = set().union(*(tomas_actores[t] for t in dia))
    print(f"Día {i}: Tomas {sorted(dia)} | Actores convocados: {sorted(actores_dia)}")

Costo mínimo optimizado: 10
Día 1: Tomas [1, 2, 3] | Actores convocados: [1, 2, 3, 4]
Día 2: Tomas [4, 6, 8] | Actores convocados: [2, 4, 5]
Día 3: Tomas [5, 7, 9] | Actores convocados: [1, 3, 5]


## Complejidad Algoritmo Fuerza Bruta

El paso más costoso del algorimo es la generación de todas las particiones posibles del conjunto de tomas, respetando el límite de tomas por día. Este proble está relacionado con el problema de partición restringida y no tiene un fórmula exaca simple, pero su complejidad está acotada por:
$$
O(B_n)
$$
donde $B_n$ es el número de Bell, que cuenta el número de particiones posibles sin tener en cuenta las restricciones como vimos previamente. Este número crece más rápido que exponencialmente.
Con las restricciones de máximo $k$ tomas por día, se reduce el número de particiones, pero aun así sigue creciendo más rápido que exponecialmente con el número de tomas, no tanto como $O(B_n)$ pero sí mucho más que $O(2^n)$.

Esta estrategia rápidamente se vuelve computacionalmente inviable incluso para n moderados.

## Algoritmo de Ramificación y Poda

El algoritmo de ramificación y poda construye iterativamente un calendario de grabación, añadiendo cada toma a los días existentes o a un nuevo día. Tras cada paso, calcula el coste parcial acumulado y compara con la mejor solución hasta el momemento. Si el coste actual ya supera al mejor, se descarta esa rama del árbol de búsqueda, evitando así explorar combinaciones no prometedoras. 

De esta forma, esta técnica mejora significativamente la eficiencia respecto al algoritmo de fuerza bruta, que examina todas las particiones posibles del conjunto de tomas.

### Implementación del Algoritmo

In [11]:
# Variables globales
mejor_coste = float('inf')
mejor_solucion = None

# Algoritmo de Ramificación y Poda
def ramificacion_y_poda(tomas_restantes, calendario_actual):
    global mejor_coste, mejor_solucion

    if not tomas_restantes:
        coste_actual = calcular_coste(calendario_actual)
        if coste_actual < mejor_coste:
            mejor_coste = coste_actual
            mejor_solucion = copy.deepcopy(calendario_actual)
        return

    toma = tomas_restantes.pop()

    # Añadir a días existentes
    for i in range(len(calendario_actual)):
        if len(calendario_actual[i]) < MAX_TOMAS_POR_DIA:
            calendario_actual[i].append(toma)
            coste_parcial = calcular_coste(calendario_actual)
            if coste_parcial < mejor_coste:
                ramificacion_y_poda(tomas_restantes, calendario_actual)
            calendario_actual[i].pop()

    # Crear nuevo día
    calendario_actual.append([toma])
    coste_parcial = calcular_coste(calendario_actual)
    if coste_parcial < mejor_coste:
        ramificacion_y_poda(tomas_restantes, calendario_actual)
    calendario_actual.pop()

    tomas_restantes.add(toma)

### Conjunto de datos de prueba

Vamos a usar el mismo conjunto de datos de prueba que para el algorimo de fuerza bruta, de esta forma podemos realizar una comparación directa de rendimiento.

In [12]:
tomas_actores = {
    1: {1, 2},
    2: {2, 3},
    3: {3, 4},
    4: {4, 5},
    5: {1, 5},
    6: {2, 4},
    7: {1, 3},
    8: {2, 5},
    9: {3, 5},
}

MAX_TOMAS_POR_DIA = 3
NUM_ACTORES = 5

In [13]:
# Ejecutar algoritmo
todas_tomas = set(tomas_actores.keys())
%time ramificacion_y_poda(todas_tomas, [])

CPU times: total: 0 ns
Wall time: 1.51 ms


In [14]:
# Mostrar resultados finales
print("Costo mínimo optimizado:", mejor_coste)
for i, dia in enumerate(mejor_solucion, start=1):
    actores_dia = set().union(*(tomas_actores[t] for t in dia))
    print(f"Día {i}: Tomas {sorted(dia)} | Actores convocados: {sorted(actores_dia)}")

Costo mínimo optimizado: 10
Día 1: Tomas [1, 2, 3] | Actores convocados: [1, 2, 3, 4]
Día 2: Tomas [4, 6, 8] | Actores convocados: [2, 4, 5]
Día 3: Tomas [5, 7, 9] | Actores convocados: [1, 3, 5]


Comparando el algorimo de fuerza bruta con el de ramificación y poda para este conjunto de datos, vemos que se obtiene el mismo resultado, lo que hace indicar que la implemntación es correcta. Sin embargo, vemos que el rendimiento del algoritmo de ramificación y poda es significativamente mejor que el de fuerza bruta.

**Tiempos de ejecución:**
- Fuerza Bruta: 21.2 s
- Ramificación y Poda: 15.6 ms

## Complejidad Algortimo Ramificación y Poda

Para estimar la complejidad del algoritmo comenzamos ignorando inicialmente la poda. Supongamos que hay $n$ tomas para grabar. En cada paso, para una toma dada, se puede asignarla a uno de los días ya existentes o crear un nuevo día con esa toma. Teniendo como máximo $n$ días en el peor caso

Por tanto, en el peor de los casos, cada una de las $n$ tomas tiene hasta $n$ posibles ubicaciones, lo que da una cota superior de complejidad:

$$
O(n^n)
$$

Este crecimiento es exponencial y, por tanto, inabordable para valores grandes de $n$.


Sin embargo, al añadir la poda y descargar las ramas que vayan superando el mejor coste encontrado puediendo reducir drásticamente el espacio de búsqueda. En el peor caso, la poda no logra eliminar muchas ramas y el algoritmo sigue teniendo complejidad exponencial. Pero en la práctica, el algoritmo suele ser mucho más eficiente, sobre todo si se encuentra pronto una buena solución temprana y se podan muchas ramas.

## Aplicacion del Algoritmo sobre los Datos del Problema

Ahora que ya hemos probado y funciona el algoritmo de ramificación y poda sobre un conjunto de prueba reducido vamos a ponerlo en práctica sobre el conjunto dado en el enunciado. Tenemos:

In [15]:
tomas_actores = {
    1: {1, 2, 3, 4, 5},
    2: {3, 4, 5},
    3: {2, 5, 7},
    4: {1, 2, 7, 8},
    5: {2, 4, 8},
    6: {1, 2, 4, 5},
    7: {1, 2, 4, 5},
    8: {1, 2, 6},
    9: {1, 2, 4},
    10: {1, 2, 6, 9},
    11: {1, 2, 3, 5, 8},
    12: {1, 2, 3, 4, 6},
    13: {1, 4, 5},
    14: {1, 3, 6},
    15: {1, 2, 7},
    16: {4, 10},
    17: {1, 3},
    18: {3, 6},
    19: {1, 3},
    20: {1, 3, 4, 5},
    21: {6, 8},
    22: {1, 2, 3, 4},
    23: {1, 3},
    24: {3, 6},
    25: {1, 2, 4, 10},
    26: {1, 3, 5, 9},
    27: {4, 5},
    28: {1, 4},
    29: {1, 5, 6},
    30: {1, 4},
}

In [17]:
# Constantes
MAX_TOMAS_POR_DIA = 6
NUM_ACTORES = 10

In [18]:
# Variables globales
mejor_coste = float('inf')
mejor_solucion = None

In [19]:
# Ejecutar algoritmo
todas_tomas = set(tomas_actores.keys())
ramificacion_y_poda(todas_tomas, [])



In [20]:
# Mostrar resultados finales
print("Costo mínimo optimizado:", mejor_coste)
for i, dia in enumerate(mejor_solucion, start=1):
    actores_dia = set().union(*(tomas_actores[t] for t in dia))
    print(f"Día {i}: Tomas {sorted(dia)} | Actores convocados: {sorted(actores_dia)}")

Costo mínimo optimizado: 27
Día 1: Tomas [1, 2, 3, 4, 11, 15] | Actores convocados: [1, 2, 3, 4, 5, 7, 8]
Día 2: Tomas [5, 8, 16, 21, 25, 28] | Actores convocados: [1, 2, 4, 6, 8, 10]
Día 3: Tomas [6, 7, 9, 13, 27, 30] | Actores convocados: [1, 2, 4, 5]
Día 4: Tomas [14, 17, 18, 19, 23, 24] | Actores convocados: [1, 3, 6]
Día 5: Tomas [10, 12, 20, 22, 26, 29] | Actores convocados: [1, 2, 3, 4, 5, 6, 9]


El tiempo de ejecución del algoritmo ha sido aproximadamente un poco más de 3 días completos. Aunque aplicando el algoritmo de ramificación y poda hemos podido alcanzar la solución que hubiera sido inviable mediante el algoritmo de fuerza bruta, el tiempo involucrado en el proceso ha sido bastante alto.

## Mejorando el Algoritmo

Para mejorar el algoritmo de ramificación y poda se pueden implementar varias optimizaciones para reducir el espacio de búsqueda y ofrecer mejor rendimiento:
1. Heurística voraz inicial: Con esto se consigue acotar desde el principio construyendo una solución inicial rápida de forma que se tenga un primer valor de coste que se usa como umbral de poda para el algortimo, eliminando muchas ramas desde el inicio.
2. Ordenación inicial: Se procesan primero las tomas que requieren más actores, ya que son más dificiles de ubicar.
3. Selección de la toma más restrictiva en cada paso: Explorando las opciones más costosas antes, si se obtiene una solución mala se descarta más rápido.

### Implementación de la versión mejorada

In [21]:
# Solución inicial rápida
def heuristica_voraz(tomas_ordenadas):
    calendario = []
    for toma in tomas_ordenadas:
        añadido = False
        for dia in calendario:
            if len(dia) < MAX_TOMAS_POR_DIA:
                dia.append(toma)
                añadido = True
                break
        if not añadido:
            calendario.append([toma])
    return calendario

In [22]:
# Ramificación y poda mejorada
def ramificacion_y_poda_mejorada(tomas_restantes, calendario_actual):
    global mejor_coste, mejor_solucion

    if not tomas_restantes:
        coste_actual = calcular_coste(calendario_actual)
        if coste_actual < mejor_coste:
            mejor_coste = coste_actual
            mejor_solucion = copy.deepcopy(calendario_actual)
        return

    # Seleccionamos la siguiente toma (más restrictiva)
    toma = max(tomas_restantes, key=lambda x: len(tomas_actores[x]))
    tomas_restantes.remove(toma)

    # Probar añadir a días existentes
    for i in range(len(calendario_actual)):
        if len(calendario_actual[i]) < MAX_TOMAS_POR_DIA:
            calendario_actual[i].append(toma)
            coste_parcial = calcular_coste(calendario_actual)
            if coste_parcial < mejor_coste:
                ramificacion_y_poda(tomas_restantes, calendario_actual)
            calendario_actual[i].pop()

    # Probar crear nuevo día
    calendario_actual.append([toma])
    coste_parcial = calcular_coste(calendario_actual)
    if coste_parcial < mejor_coste:
        ramificacion_y_poda(tomas_restantes, calendario_actual)
    calendario_actual.pop()

    tomas_restantes.add(toma)

In [23]:
# Preparar orden: tomas con más actores primero
tomas_ordenadas = sorted(tomas_actores.keys(), key=lambda x: -len(tomas_actores[x]))

# Solución inicial para acotar
mejor_solucion = heuristica_voraz(tomas_ordenadas)
mejor_coste = calcular_coste(mejor_solucion)

In [29]:
# Ejecutar búsqueda
%time ramificacion_y_poda(set(tomas_actores.keys()), [])

CPU times: total: 20min 39s
Wall time: 20min 55s


In [30]:
# Mostrar resultados finales
print("Costo mínimo optimizado:", mejor_coste)
for i, dia in enumerate(mejor_solucion, start=1):
    actores_dia = set().union(*(tomas_actores[t] for t in dia))
    print(f"Día {i}: Tomas {dia} | Actores convocados: {sorted(actores_dia)}")

Costo mínimo optimizado: 27
Día 1: Tomas [1, 11, 12, 6, 10, 26] | Actores convocados: [1, 2, 3, 4, 5, 6, 8, 9]
Día 2: Tomas [4, 8, 3, 15, 29, 21] | Actores convocados: [1, 2, 5, 6, 7, 8]
Día 3: Tomas [7, 20, 22, 2, 13, 27] | Actores convocados: [1, 2, 3, 4, 5]
Día 4: Tomas [25, 9, 5, 16, 30, 28] | Actores convocados: [1, 2, 4, 8, 10]
Día 5: Tomas [14, 17, 18, 19, 23, 24] | Actores convocados: [1, 3, 6]


Con estas mejoras hemos pasado de alcanzar al solución del problema en 3 días de cálculo a tan sólo 20 minutos.

## Referencias



\[1] Wikipedia, "Número de Bell," *Wikipedia, La enciclopedia libre*. Disponible en: [https://es.wikipedia.org/wiki/N%C3%BAmero\_de\_Bell](https://es.wikipedia.org/wiki/N%C3%BAmero_de_Bell). 

\[2] R. Guerequeta y A. Vallecillo, *Técnicas de diseño de algoritmos*, capítulo 7, 2000. Disponible en: [http://www.lcc.uma.es/%7eav/Libro/CAP7.pdf](http://www.lcc.uma.es/%7eav/Libro/CAP7.pdf). 


